In [2]:
import nltk
from bs4 import BeautifulSoup
import requests
import numpy as np
from nltk.corpus import cmudict
pronounciations = cmudict.dict()
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [38]:
# function to pull text from NPR's plain text website
def NPR_text(id_number):
    html_page = requests.get('https://text.npr.org/s.php?sId={}'.format(id_number))
    soup = BeautifulSoup(html_page.content, 'html.parser')
    text = soup.text.split('\n')[15:-10]
    tokens = []
    for block in text:
        tokens += nltk.word_tokenize(block)
    #the sklearn vectorizer's stopwords removal is ... iffy. So I'm removing stopwords here
    stop_words = list(set(stopwords.words('english'))) + ['npr'] 
    text_words = [word for word in tokens[7:] if word.lower() not in stop_words]
    return text_words

In [22]:
#NPR has a convenient plain text site, but no good plain text index of articles, and the id numbers are not
#sequential. Unfortunately, this is therefore a hand compiled list of article ids, which proved to be the 
# real sticking part of this part of the project. I would have prefered ten times as much text, but I'm going to save
#that for the final product, once I have a good way to automate it all
ids = [738767035,738146040,733487137,738228904,737844386,738736102,738073028,738071454,738522707,738719658,
       738531267,738422258,738426933,738385096,738569508,738573237,738424247,737754949,738062312,738059913,
       737976138,738030272,737983143,737797744,737911832,735506637,737662103,738001096,737114153,736037164,
       734810718,732052345,738766431,738139123,737666425,737535442,737613627,736020514,738283044,738791698,
       738422439,738216499,737739602,738868364,738164642,739026295,738938037,738939875,738970664,738894705,
       736573450,738905306,731346268,738283044,738624184,738766431,739123627,738988815,739033701,738965440,
       738458515,738860155,721912974,709979787,737475179,737429007,736940431,734525036,737259955,736945875,
       737041530,737004547,736922360,736993800,736946832,736910015,736612462,736824783,736644485,736572732,
       735429939,736330111,736290295,728822381,728590535,728611992,728370966,728559903,728334635,728498303,
       728566814,728576269,727945689,728283462,728059679,728526221,728484981,728356084,728393644,728257932,
       728229142,728079493,728089038,727914874,728111494,728141835,727849422,582724398,727842244,727847695,
       727832275,727807874,727596397,727780350,722875100,727497208,725755061,727702957,731175020,726754608,
       725064362,721017931,708160805,705128363,706894334,706884510,706888445,707561940,706909807,706869166,
       702340275,706513097,706499110,704743237,705072849,704901636,698732975,698736932,698656874,698646232,
       698543551,697160095,697699971,698622827,698126773,696814552,697599611,696272090,697024516,696284474,
       694233506,695962061,695929854,695175058,727439670,726519660,727179915,727154492,723983713,726887624,
       726884720,726817332,726759993,726803919,726680414,739171142,739260129,739185742,725696057,725428447,
       739224759,739198317,726456823,726381391,726349559,726353723,726117690,725951972,725903534,725702479,
       725668115,725026140,725652420,725439880,725113252,725038799,724978646,724951238,724910121,724309081,
       724274982,723193259,724133556,724164873,723924403,712492409,723888420,723282022,699195211,699143911,
       740009105,734430818,741103503,740505613,738594345,739043318,739530731,741326915,739967250,741084295,
       734430818,741326915,740714662,741008967,741098605,740567368,740847647,741307427,740817989,73661770,
       739513902,740490132,740667457,740257654,735314929
    ]

In [39]:
# and here's an article not in the corpus compiled above, it is about Hurricane Barry
test_article = NPR_text(741382999)

In [33]:
#Create a corpus of actual text data with the ids
texts = []
for number in ids:
    texts.append(' '.join(NPR_text(number)))
    

In [35]:
#sklearn makes it easy to vectorize and perform LDA:
cv = CountVectorizer(max_df=.8, min_df=3, stop_words='english')
vectors = cv.fit_transform(texts)
LDA = LatentDirichletAllocation(n_components=10)
LDA.fit(vectors)

#And to demonstrate what the topics it finds look like:
for index,topic in enumerate(LDA.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['deal', 'like', 'time', 'acosta', 'million', 'beer', 'rice', 'years', 'new', 'coffee', 'census', 'iran', 'said', 'says', 'epstein']


THE TOP 15 WORDS FOR TOPIC #1
['year', 'weight', 'anne', 'cancer', 'women', 'work', 'queen', 'like', 'new', 'years', 'best', 'study', 'tea', 'people', 'says']


THE TOP 15 WORDS FOR TOPIC #2
['kind', 'school', 'year', 'new', 'want', 'city', 'women', 'time', 'going', 'know', 'life', 'think', 'like', 'people', 'says']


THE TOP 15 WORDS FOR TOPIC #3
['national', 'government', 'year', 'asylum', 'migrants', 'north', 'people', 'npr', 'president', 'new', 'immigration', 'country', 'trump', 'mexico', 'said']


THE TOP 15 WORDS FOR TOPIC #4
['000', 'water', 'say', 'time', 'says', 'city', 'border', 'government', 'reported', 'year', 'people', 'told', 'according', 'officials', 'said']


THE TOP 15 WORDS FOR TOPIC #5
['power', 'environmental', 'car', 'energy', 'electric', 'sexual', 'family', 'louis', 'people', 'says', 'cancer', 'said', 

My next notion is operationalize LDA analysis: feed in a new text, see what 'topic' it belongs to and return the words from within the text that are most weighted to that topic.

This function prints out a topic number so you can see for yourself which topic it falls into, but, it crucially also returns a list of words. My thought is that you can then use only the highly weighted words to create a topical haiku that will (hopefully) be more aligned with the topic than randomly chosen words would be (since I weighted the random choosing of words in my basic haiku make by how often the words appeared in the text, I am not actually convinced this will produce more suggestive results at all).

In [36]:
def key_topics(text, vectorizer, LDA, number_words=10): 
    vector = vectorizer.transform(text)
    topic_index = LDA.transform(vector)[0].argmax() #topic number 1, or 2, etc.
    positions = [] #for the index positions of the words from the text in matrix that is all corpus words
    for index in vector.todense():
        try: #matrix will have rows of all zeroes if a word is a stopword or otherwise removed by the vectorizer
            positions.append(index.nonzero()[1][0])
        except:
            pass
    positions = list(set(positions))
    lda_weights = []
    for n in positions:
        lda_weights.append(LDA.components_[topic_index][n]) #the component is the relative weight within the LDA model
    lda_weights = np.array(lda_weights)
    word_indices = lda_weights.argsort()[-number_words:] #Find the words with the largest weights within the topic
    words = [vectorizer.get_feature_names()[positions[n]] for n in word_indices]
    print(f'Topic number {topic_index}')
    return words

How does this work in practice? Ok. The test article was a news article, about Hurrican Barry, and the LDA model is smart enough to realize it's a news article (the fact that Trump's name appears in it probably helped). If you look through the words that are most weighted towards the topic cluster it identifies you see clear examples of 'news-speak': 'residents', 'reports', 'officials', etc.

Still, there are some quirks: for one thing, it identifies the day 'Thursday' as significant. Maybe this actually makes sense! Maybe news articles (rather than articles from the arts section) are more likely to specify a particular day of the week. It also seems to me that some words might point the model to a particular topic cluster for the 'wrong' reasons. The word 'water', for instance. Knowing a bit about the articles that went into this model, it doesn't surprise me that water might be clustered with Trump and other news related words: There have been numerous articles over the summer about the detention centers on the southern border, including discussions of whether the people shamefully imprisoned therein have access to clean water. Yet, in the context of this test article, it's a different sort of water, rain/flood water, that's being used to link the article to this topic cluster.

In [40]:
key_topics(test_article, cv, LDA, 20)

Topic number 4


['agency',
 'power',
 'nearly',
 'thursday',
 'department',
 'national',
 'reports',
 'center',
 'president',
 'new',
 'trump',
 '000',
 'water',
 'say',
 'says',
 'city',
 'people',
 'told',
 'officials',
 'said']

Now to integrate this into the haiku generator: I will use this analyzer to determine a list of words to use. First I will use my previous subject-finder function to determine the 'subject', but all other words will be chosen from this smaller list.

For clarity, and since this is a sufficiently short project, I'm going to simply copy in the key functions below, rather than importing from a seperate file. (Also, the find topic function has been adjusted to taken in the list of tokens without stopwords I've generated in this notebook).

For more info on the process of these functions, please see the notebook in the topical haiku folder.

In [47]:
def find_names(text_words, nnps):
    text_bigrams = list(nltk.bigrams(text_words))
    names = [bigram for bigram in text_bigrams if bigram[0] in nnps
             and bigram[1] in nnps]
    return names

def find_topic(text_words):
    text_pos = nltk.pos_tag(text_words)

    text_pos_minus_stop = [(word, pos) for (word,pos) in text_pos]
    NNPs = [word for (word,pos) in text_pos_minus_stop if pos=='NNP'] #NNP is just 'proper nouns'
    names = find_names(text_words, NNPs)
    most_common = nltk.FreqDist(NNPs).most_common(1)[0][0] 
    #NLTK has a convenient, built in frequency dictionary function
    
    #now to check if the most common proper noun is in a name, either first or last name
    if most_common in [name1 for (name1,name2) in names]:
        for name in names:
            if most_common == name[0]:
                return name
    if most_common in [name2 for (name1,name2) in names]:
        for name in names:
            if most_common == name[1]:
                return name
    return most_common

In [48]:
find_topic(test_article)

'Louisiana'

In [49]:
def syllable_count(word):
    count = 0
    pron = pronounciations[word.lower()][0]
    for syl in pron:
        if syl[-1].isdigit():
            count +=1
    return count

def find_nouns_adjs(text_words):
    noun_adj_tags = ['NN','NNS','JJ','JJR','JJS']
    noun_adjs = [word for (word,pos) in text_words if pos in noun_adj_tags]
    wnl = nltk.WordNetLemmatizer()
    lemmas = [wnl.lemmatize(word) for word in noun_adjs]
    return lemmas

def create_syllable_lists(lemmas):
    syllables = {1:[],2:[],3:[],4:[],5:[]}
    freq_dict =nltk.FreqDist(lemmas)
    lemmas = list(set(lemmas))
    for word in lemmas:
        try:
            syllables[syllable_count(word)].append((word,freq_dict[word]))
        except:
            pass
    return syllables

In [50]:
def rand_word(cfd):
    cum_dic = {}
    count = 0
    if len(cfd) == 1:
        return cfd[0][0]
    for word in cfd:
        count += word[1]
        cum_dic[word[0]] = count
    rand_int = np.random.randint(1,count+1)
    for word in cum_dic:
        if cum_dic[word] > rand_int:
            return word
        
def fill_out_line(num_syllables, syllable_dict, line = [], already_used=[]):
    while num_syllables > 1:
        find_syl = np.random.randint(1,6)
        if find_syl <= num_syllables and len(syllable_dict[find_syl]) >0:
            words = syllable_dict[find_syl]
            new_word = rand_word(words)
            if new_word not in already_used: #make sure we don't use the same word more than once
                line.append(new_word)
                already_used.append(new_word)
            num_syllables -= find_syl
    if num_syllables == 1:
        words = syllable_dict[1]
        new_word = rand_word(words)
        if new_word not in already_used:
            line.append(rand_word(words))
            already_used.append(new_word)
    return line, already_used #also return the already used list so we can use the same list across multiple lines

def write_haiku(id_num, cv, LDA, num_words):
    text = NPR_text(id_num)
    topic = find_topic(text)
    list_of_words = key_topics(text, cv, LDA, num_words)
    word_list_pos = nltk.pos_tag(list_of_words)
    
    noun_adjs = find_nouns_adjs(word_list_pos)
    syllable_dict = create_syllable_lists(noun_adjs)
    
    if type(topic) !=str:
        topic_syls = syllable_count(topic[0]) + syllable_count(topic[1])
    else:
        topic_syls = syllable_count(topic)
    line_1 = []
    line_2 = []
    line_3 = []
    already_used = []
    if topic_syls <=5:
        if type(topic) !=str:
            line_1 += [word for word in topic]
            already_used += topic[0],topic[1]
        else:
            line_1.append(topic)
            already_used.append(topic)
        line_1, already_used = fill_out_line(5-topic_syls, syllable_dict, line_1, already_used)
        line_2, already_used = fill_out_line(7, syllable_dict, line_2, already_used)
        line_3, already_used = fill_out_line(5, syllable_dict, line_3, already_used)
    elif topic_syls <8:
        if type(topic) !=str:
            line_2 += [word for word in topic]
            already_used += topic[0],topic[1]
        else:
            line_2.append(topic)
            already_used.append(topic)
        line_1, already_used = fill_out_line(5, syllable_dict, line_1, already_used)
        line_2, already_used = fill_out_line(7-topic_syls, syllable_dict, line_2, already_used)
        line_3, already_used = fill_out_line(5, syllable_dict, line_3, already_used)

    return line_1, line_2, line_3

In [51]:
write_haiku(741382999, cv, LDA, 30)

Topic number 4


(['Louisiana'], ['national', 'day', 'department'], ['emergency', 'day'])

In [53]:
write_haiku(741382999, cv, LDA, 40)

Topic number 4


(['Louisiana'], ['new', 'emergency', 'water'], ['central', 'report', 'day'])

Trying another article. This one is about the new White House press secretary, Stephanie Grisham.

In [59]:
write_haiku(740505613, cv, LDA, 40)

Topic number 0


(['Grisham', 'spokesperson'],
 ['secretary', 'good', 'new'],
 ['order', 'president'])

In [65]:
write_haiku(740505613, cv, LDA, 50)

Topic number 0


(['Grisham', 'spokesperson'],
 ['biggest', 'administration'],
 ['white', 'medium', 'good'])

Does this process improve my haiku generating? Not really. If anything, it makes it worse for a couple of reasons. My previous generator used words in proportion to their frequency in the text and didn't throw out any words except for stop words. Now, however, the text words are passed through a vectorizer that throws out words that don't appear frequently enough in all the texts of the corpus and is using weights that are influenced by the model/corpus overall. As a result, words that are relatively particular to a given article might not make it into the list of words to be used, even though those are exactly the sorts of words you want to appear in the summary haiku!

In the long run of the project a strategy might be necessary to unlock a wider dictionary of words to use. If you're creating a limerick you may need to look for words that don't appear in the text in order to find rhymes or to generate new sentences that scan properly. How will the computer know which sorts of words make sense? This might be one way to group words: 'I see this article falls into this topic cluster, other words associated with this cluster are...' It's clear to me, however, that in order to use this sort of strategy, you just need more: more text, and more diverse text, that can be grouped into a greater number of topic clusters.